In [1]:
import pandas as pd
import re
import numpy as np
from functools import reduce
import pandas as pd
import seaborn as sns
from igel import Igel

In [2]:
def clean_dataset_int(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df[indices_to_keep].astype(np.float64)

def clean_final_dataset(df):
    df = df.drop(['category','eventID','eventType','eventID'],axis =1)
    return df

def merge_datasets(df):
    dfs = [dataset_labelled, df]
    df_final = reduce(lambda left,right: pd.merge(left,right,on='tweet_id'), dfs)
    return df_final

def clear_res(res):
    res['tweet_id'] = pd.to_numeric(res['tweet_id'].apply(lambda x: re.sub(',', '.', str(x))))
    del res['numb_of_mentions']
    del res['numb_of_urls']
    del res['numb_of_hashtags']
    del res['numb_of_personal_pronouns']
    del res['numb_of_present_tenses']
    del res['positive_sentiment']
    del res['negative_sentiment']
    del res['numb_of_questions']
    del res['tweet_length']
    del res['userFollowersCount']
    del res['userFriendsCount']
    del res['user_list_count']
    del res['dict_precision']
    del res['dict_recall']
    del res['dict_f_measure']
    return res

def clear_vec(vec):
    del vec['positive_sentiment']
    del vec['negative_sentiment']
    del vec['numb_of_questions']
    del vec['tweet_length']
    del vec['userFollowersCount']
    del vec['userFriendsCount']
    del vec['user_list_count']
    del vec['dict_precision']
    del vec['dict_recall']
    del vec['dict_f_measure']
    return vec

def trim(df):
    df['timestamp'] = df['timestamp'].str[1:]
    df['timestamp'] = df['timestamp'].str[:-1]
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df
    
    
def clean_brackets(df):
    df['tweet_id'] = df['tweet_id'].str[:-1]
    df['positive_sentiment'] = df['positive_sentiment'].str[:-1]
    df['negative_sentiment'] = df['negative_sentiment'].str[:-1]
    df['numb_of_questions'] = df['numb_of_questions'].str[:-1]
    df['tweet_length'] = df['tweet_length'].str[:-1]
    df['userFollowersCount'] = df['userFollowersCount'].str[:-1]
    df['userFriendsCount'] = df['userFriendsCount'].str[:-1]
    df['user_list_count'] = df['user_list_count'].str[:-1]
    df['dict_precision'] = df['dict_precision'].str[:-1]
    df['dict_recall'] = df['dict_recall'].str[:-1]
    df['dict_f_measure'] = df['dict_f_measure'].str[:-1]
    df['numb_of_media'] = df['numb_of_media'].str[:-1]
    #df.tweet_id.map(lambda x: '{:.0f}'.format(x))
    #df.tweet_id = df.tweet_id.map(lambda x: '{:.0f}'.format(x))
    df['tweet_id'] = pd.to_numeric(df['tweet_id'].apply(lambda x: re.sub(',', '.', str(x))))
    df['tweet_id'].describe()
    return df

def clean_float(df):
    # Obj -> Str -> float   
    pd.set_option('display.float_format', '{:.2f}'.format)
    for col in df.columns.values:
        print(col)
        df[col] = df[col].astype(str).astype(float)
    float_col = df.select_dtypes(include=['float64'])
    for col in float_col.columns.values:
        df[col] = df[col].astype('int64')
    display(df.dtypes)
    return df

def clean_initial_feature_vector(df):
    df = trim(df)
    df = clean_brackets(df)
    return df

In [3]:
# Set the column names
feature_vector_keys = ["timestamp", "tweet_id", "positive_sentiment", "negative_sentiment", \
                       "numb_of_mentions", "numb_of_urls", "numb_of_media","numb_of_hashtags", \
                       "numb_of_personal_pronouns", "numb_of_present_tenses", "numb_of_past_tenses", \
                       "sent_from_web", "numb_of_weird_chars", "numb_of_questions", "numb_of_emoticons", \
                       "numb_of_swearing_words", "numb_of_slang_words", "numb_of_intensifiers", \
                       "tweet_length", "userFollowersCount","userFriendsCount", "user_numb_of_tweets",\
                       "user_list_count", "tfidf_fire", "dict_precision", "dict_recall", "dict_f_measure"
                       ]

# The Topic IDs that we'll map
eventidTopicidMap =	{
  "athensEarthquake2020": "TRECIS-CTIT-H-Test-035",
  "baltimoreFlashFlood2020": "TRECIS-CTIT-H-Test-036",
  "brooklynBlockPartyShooting2020": "TRECIS-CTIT-H-Test-037",
  "daytonOhioShooting2020": "TRECIS-CTIT-H-Test-038",
  "elPasoWalmartShooting2020": "TRECIS-CTIT-H-Test-039",
  "gilroygarlicShooting2020": "TRECIS-CTIT-H-Test-040",
  "hurricaneBarry2020": "TRECIS-CTIT-H-Test-041",
  "indonesiaEarthquake2020": "TRECIS-CTIT-H-Test-042",
  "keralaFloods2020": "TRECIS-CTIT-H-Test-043",
  "myanmarFloods2020": "TRECIS-CTIT-H-Test-044",
  "papuaNewguineaEarthquake2020": "TRECIS-CTIT-H-Test-045",
  "siberianWildfires2020": "TRECIS-CTIT-H-Test-046",
  "typhoonKrosa2020": "TRECIS-CTIT-H-Test-047",
  "typhoonLekima2020": "TRECIS-CTIT-H-Test-048",
  "whaleyBridgeCollapse2020": "TRECIS-CTIT-H-Test-049"
}

priority_scorer = {
    '1.0' : 'Critical',
    '0.9' : 'Critical',
    '0.8' : 'High',
    '0.7' : 'High',
    '0.6' : 'High',
    '0.5' : 'Medium',
    '0.4' : 'Medium',
    '0.2' : 'Low',
    '0.3' : 'Medium',
    '0.0' : 0.0,
}

categoriser = {
    0 : 'Advice',
    1.0 : 'CleanUp',
    2 : 'ContextualInformation',
    3 : 'Discussion',
    4 : 'Donations',
    5 : 'EmergingThreats',
    6 : 'Factoid',
    7 : 'FirstPartyObservation',
    8 : 'GoodsServices',
    9 : 'Hashtags',
    10 : 'InformationWanted',
    11 : 'Irrelevant',
    12.0 : 'Location',
    13 : 'MovePeople',
    14 : 'MultimediaShare',
    15 : 'NewSubEvent',
    16 : 'News',
    17 : 'Official',
    18 : 'OriginalEvent',
    19 : 'SearchAndRescue',
    20 : 'Sentiment',
    21 : 'ServiceAvailable',
    22 : 'ThirdPartyObservation',
    23 : 'Volunteer',
    24 : 'Weather',
}



In [4]:
# Currently loading the feature vector from a .txt file exported from Play
feature_vector_input = pd.read_csv("../../../0-data/processed/all_new.txt", sep=" ", header=None)

# Set the column names
feature_vector_input.columns = feature_vector_keys

# Group by tweet
feature_vector_input = feature_vector_input.groupby(['tweet_id']).agg('first')

# Reset the index
feature_vector_input.reset_index(level=0, inplace=True)

# Clean & drop empty rows
feature_vector_input = clean_initial_feature_vector(feature_vector_input)

feature_vector_input

,tweet_id,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,numb_of_personal_pronouns,numb_of_present_tenses,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1.157803e+18,1.564876819E12,2.0,66.0,"2.0,","0.0,",0.0,"0.0,","0.0,","0.0,",...,"0.0,",68.0,70.0,140.0,"0.0,",1.0,"0.0,",0.0,0.0,0.0
1,1.157805e+18,1.56487711E12,12.0,48.0,"0.0,","1.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",96.0,1445.0,4608.0,"0.0,",6.0,"0.0,",0.0,0.0,0.0
2,1.157807e+18,1.564877673E12,1.0,68.0,"1.0,","0.0,",0.0,"0.0,","0.0,","0.0,",...,"0.0,",51.0,1288.0,3510.0,"0.0,",33.0,"0.0,",0.0,0.0,0.0
3,1.157807e+18,1.564877679E12,6.0,63.0,"0.0,","0.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",61.0,1349.0,1533.0,"0.0,",23.0,"0.0,",0.0,0.0,0.0
4,1.157813e+18,1.564879041E12,27.0,23.0,"1.0,","0.0,",0.0,"0.0,","0.0,","0.0,",...,"0.0,",67.0,130.0,148.0,"0.0,",2.0,"0.0,",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,1.158887e+18,1.565135085E12,12.0,48.0,"0.0,","0.0,",0.0,"0.0,","0.0,","0.0,",...,"0.0,",272.0,1.0,1.0,"0.0,",0.0,"0.0,",0.0,0.0,0.0
468,1.158888e+18,1.565135502E12,5.0,60.0,"0.0,","0.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",120.0,92.0,306.0,"0.0,",6.0,"0.0,",0.0,0.0,0.0
469,1.158888e+18,1.565135517E12,3.0,23.0,"0.0,","0.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",131.0,85.0,2.0,"0.0,",2.0,"0.0,",0.0,0.0,0.0
470,1.158889e+18,1.565135567E12,0.0,1.0,"1.0,","0.0,",0.0,"0.0,","0.0,","0.0,",...,"0.0,",57.0,642.0,946.0,"0.0,",5.0,"0.0,",0.0,0.0,0.0


In [5]:
# Create a numeric version for our model
df_int = clean_dataset_int(feature_vector_input)
df_int

,tweet_id,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,numb_of_personal_pronouns,numb_of_present_tenses,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1.157803e+18,1.564877e+11,20.0,660.0,20.0,0.0,0.0,0.0,0.0,0.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
1,1.157805e+18,1.564877e+10,120.0,480.0,0.0,10.0,10.0,0.0,0.0,0.0,...,0.0,960.0,14450.0,46080.0,0.0,60.0,0.0,0.0,0.0,0.0
2,1.157807e+18,1.564878e+11,10.0,680.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,510.0,12880.0,35100.0,0.0,330.0,0.0,0.0,0.0,0.0
3,1.157807e+18,1.564878e+11,60.0,630.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,610.0,13490.0,15330.0,0.0,230.0,0.0,0.0,0.0,0.0
4,1.157813e+18,1.564879e+11,270.0,230.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,670.0,1300.0,1480.0,0.0,20.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,1.158887e+18,1.565135e+11,120.0,480.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2720.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0
468,1.158888e+18,1.565136e+11,50.0,600.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,1200.0,920.0,3060.0,0.0,60.0,0.0,0.0,0.0,0.0
469,1.158888e+18,1.565136e+11,30.0,230.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,1310.0,850.0,20.0,0.0,20.0,0.0,0.0,0.0,0.0
470,1.158889e+18,1.565136e+11,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,570.0,6420.0,9460.0,0.0,50.0,0.0,0.0,0.0,0.0


In [6]:
# Load the provided Labels - These are generated in 0_Labels.ipynb
dataset_labelled = pd.read_csv("../3-csv/dataset_labelled_tweets.csv")
dataset_labelled

,tweet_id,priority,category,postCategories_x,eventID,eventType
0,1128285482784366592,0.75,Location,12,papuaNewguineaEarthquake2020,Unknown
1,1128285482784366592,0.75,Factoid,12,papuaNewguineaEarthquake2020,Unknown
2,1128285665186197504,0.25,Location,12,papuaNewguineaEarthquake2020,Unknown
3,1128285665186197504,0.25,Factoid,12,papuaNewguineaEarthquake2020,Unknown
4,1128285665186197504,0.25,Hashtags,12,papuaNewguineaEarthquake2020,Unknown
...,...,...,...,...,...,...
15264,1162006062867918848,0.25,Irrelevant,16,typhoonKrosa2020,Unknown
15265,1162006062867918848,0.25,Location,16,typhoonKrosa2020,Unknown
15266,1162006062867918848,0.25,MultimediaShare,16,typhoonKrosa2020,Unknown
15267,1162006062867918848,0.25,Hashtags,16,typhoonKrosa2020,Unknown


In [7]:
# Merges the input feature vector with the labels
merged_df = merge_datasets(df_int)

# Export as Igel takes a .csv input
merged_df.to_csv("../3-csv/merged_df.csv", index=False)
merged_df

,tweet_id,priority,category,postCategories_x,eventID,eventType,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1157803414119538688,0.25,Irrelevant,11,baltimoreFlashFlood2020,Unknown,1.564877e+11,20.0,660.0,20.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
1,1157803414119538688,0.25,Irrelevant,11,baltimoreFlashFlood2020,Unknown,1.564877e+11,20.0,660.0,20.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
2,1157803414119538688,0.25,Irrelevant,11,baltimoreFlashFlood2020,Unknown,1.564877e+11,20.0,660.0,20.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
3,1157806994369187840,0.25,Irrelevant,11,baltimoreFlashFlood2020,Unknown,1.564878e+11,10.0,680.0,10.0,...,0.0,510.0,12880.0,35100.0,0.0,330.0,0.0,0.0,0.0,0.0
4,1157806994369187840,0.25,Irrelevant,11,baltimoreFlashFlood2020,Unknown,1.564878e+11,10.0,680.0,10.0,...,0.0,510.0,12880.0,35100.0,0.0,330.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,1158888683552071680,0.25,Irrelevant,11,baltimoreFlashFlood2020,Unknown,1.565136e+11,0.0,10.0,10.0,...,0.0,570.0,6420.0,9460.0,0.0,50.0,0.0,0.0,0.0,0.0
643,1158888683552071680,0.25,Irrelevant,11,baltimoreFlashFlood2020,Unknown,1.565136e+11,0.0,10.0,10.0,...,0.0,570.0,6420.0,9460.0,0.0,50.0,0.0,0.0,0.0,0.0
644,1158890152946593792,0.25,Irrelevant,11,baltimoreFlashFlood2020,Unknown,1.565136e+11,50.0,590.0,10.0,...,0.0,690.0,30980.0,41240.0,0.0,20.0,0.0,0.0,0.0,0.0
645,1158890152946593792,0.25,Irrelevant,11,baltimoreFlashFlood2020,Unknown,1.565136e+11,50.0,590.0,10.0,...,0.0,690.0,30980.0,41240.0,0.0,20.0,0.0,0.0,0.0,0.0


In [8]:
# Remove non-numeric values 
merged_df_int = clean_final_dataset(merged_df)

# Export as Igel takes a .csv input
merged_df_int.to_csv("../3-csv/merged_df_int.csv", index=False)
merged_df_int

,tweet_id,priority,postCategories_x,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1157803414119538688,0.25,11,1.564877e+11,20.0,660.0,20.0,0.0,0.0,0.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
1,1157803414119538688,0.25,11,1.564877e+11,20.0,660.0,20.0,0.0,0.0,0.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
2,1157803414119538688,0.25,11,1.564877e+11,20.0,660.0,20.0,0.0,0.0,0.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
3,1157806994369187840,0.25,11,1.564878e+11,10.0,680.0,10.0,0.0,0.0,0.0,...,0.0,510.0,12880.0,35100.0,0.0,330.0,0.0,0.0,0.0,0.0
4,1157806994369187840,0.25,11,1.564878e+11,10.0,680.0,10.0,0.0,0.0,0.0,...,0.0,510.0,12880.0,35100.0,0.0,330.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,1158888683552071680,0.25,11,1.565136e+11,0.0,10.0,10.0,0.0,0.0,0.0,...,0.0,570.0,6420.0,9460.0,0.0,50.0,0.0,0.0,0.0,0.0
643,1158888683552071680,0.25,11,1.565136e+11,0.0,10.0,10.0,0.0,0.0,0.0,...,0.0,570.0,6420.0,9460.0,0.0,50.0,0.0,0.0,0.0,0.0
644,1158890152946593792,0.25,11,1.565136e+11,50.0,590.0,10.0,0.0,0.0,0.0,...,0.0,690.0,30980.0,41240.0,0.0,20.0,0.0,0.0,0.0,0.0
645,1158890152946593792,0.25,11,1.565136e+11,50.0,590.0,10.0,0.0,0.0,0.0,...,0.0,690.0,30980.0,41240.0,0.0,20.0,0.0,0.0,0.0,0.0


In [9]:
# Remove the priority for our prediction
merged_df_wo_priority = merged_df_int

del merged_df_wo_priority['priority']
del merged_df_wo_priority['postCategories_x']

merged_df_wo_priority.to_csv("../3-csv/merged_df_wo_priority.csv", index=False)
merged_df_wo_priority

,tweet_id,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,numb_of_personal_pronouns,numb_of_present_tenses,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1157803414119538688,1.564877e+11,20.0,660.0,20.0,0.0,0.0,0.0,0.0,0.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
1,1157803414119538688,1.564877e+11,20.0,660.0,20.0,0.0,0.0,0.0,0.0,0.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
2,1157803414119538688,1.564877e+11,20.0,660.0,20.0,0.0,0.0,0.0,0.0,0.0,...,0.0,680.0,700.0,1400.0,0.0,10.0,0.0,0.0,0.0,0.0
3,1157806994369187840,1.564878e+11,10.0,680.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,510.0,12880.0,35100.0,0.0,330.0,0.0,0.0,0.0,0.0
4,1157806994369187840,1.564878e+11,10.0,680.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,510.0,12880.0,35100.0,0.0,330.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,1158888683552071680,1.565136e+11,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,570.0,6420.0,9460.0,0.0,50.0,0.0,0.0,0.0,0.0
643,1158888683552071680,1.565136e+11,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,570.0,6420.0,9460.0,0.0,50.0,0.0,0.0,0.0,0.0
644,1158890152946593792,1.565136e+11,50.0,590.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,690.0,30980.0,41240.0,0.0,20.0,0.0,0.0,0.0,0.0
645,1158890152946593792,1.565136e+11,50.0,590.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,690.0,30980.0,41240.0,0.0,20.0,0.0,0.0,0.0,0.0


In [10]:
# Select the options for ML: [cluster, hyper, iris, regres, forest, multi]
#Fit
algo_fit = 'yaml/multi.yaml'

# Evaluate
algo_eval = 'yaml/hyper.yaml'

# Predict
algo_predict = 'yaml/hyper.yaml'   

In [11]:
# Fit
params = {
        'cmd': 'fit',    # provide the command you want to use. whether fit, evaluate or predict
        'data_path': "../3-csv/merged_df_int.csv",
        'yaml_path': algo_fit
}

Igel(**params)

INFO - Entered CLI args: {'cmd': 'fit', 'data_path': '../3-csv/merged_df_int.csv', 'yaml_path': 'yaml/multi.yaml'}
INFO - Executing command: fit ...
INFO - reading data from ../3-csv/merged_df_int.csv
INFO - You passed the configurations as a yaml file.
INFO - your chosen configuration: {'dataset': {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default'}}, 'model': {'type': 'regression', 'algorithm': 'RandomForest'}, 'target': ['priority', 'postCategories_x']}
INFO - dataset_props: {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default'}} 
model_props: {'type': 'regression', 'algorithm': 'RandomForest'} 
 target: ['priority', 'postCategories_x'] 

INFO - dataset shape: (647, 29)
INFO - dataset attributes: ['tweet_id', 'priority', 'postCategories_x', 'timestamp', 'positive_sentiment', 'negative_sentiment', 'numb_of_mentions', 'numb_of_urls', 'numb_of_media', 'numb_of_hashtags', 'numb_of_personal_pronouns', 'numb_of_present_tenses', 'numb_of_past_tenses', 'sent_fr

In [12]:
# Evaluate
params = {
        'cmd': 'evaluate',    # provide the command you want to use. whether fit, evaluate or predict
        'data_path': "../3-csv/merged_df_int.csv",
        'yaml_path': algo_eval
}

Igel(**params)

INFO - Entered CLI args: {'cmd': 'evaluate', 'data_path': '../3-csv/merged_df_int.csv', 'yaml_path': 'yaml/hyper.yaml'}
INFO - Executing command: evaluate ...
INFO - reading data from ../3-csv/merged_df_int.csv
INFO - path of the pre-fitted model => /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results/model.sav
INFO - result path: /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results 
INFO - loading model form /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results/model.sav 
INFO - dataset shape: (647, 29)
INFO - dataset attributes: ['tweet_id', 'priority', 'postCategories_x', 'timestamp', 'positive_sentiment', 'negative_sentiment', 'numb_of_mentions', 'numb_of_urls', 'numb_of_media', 'numb_of_hashtags', 'numb_of_personal_pronouns', 'numb_of_present_tenses', 'numb_of_past_tenses', 'sent_from_web', 'numb_of_weird_chars', 'numb_of_questions', 'numb_of_emoticons', 'numb_of_swearing_words', 'numb_of_slang_words', 'numb_of

In [13]:
# Predict
params = {
        'cmd': 'predict',    # provide the command you want to use. whether fit, evaluate or predict
        'data_path': "../3-csv/merged_df_wo_priority.csv",
        'yaml_path': algo_predict
}

Igel(**params)

INFO - Entered CLI args: {'cmd': 'predict', 'data_path': '../3-csv/merged_df_wo_priority.csv', 'yaml_path': 'yaml/hyper.yaml'}
INFO - Executing command: predict ...
INFO - reading data from ../3-csv/merged_df_wo_priority.csv
INFO - path of the pre-fitted model => /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results/model.sav
INFO - loading from /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results/model.sav
INFO - dataset shape: (647, 27)
INFO - dataset attributes: ['tweet_id', 'timestamp', 'positive_sentiment', 'negative_sentiment', 'numb_of_mentions', 'numb_of_urls', 'numb_of_media', 'numb_of_hashtags', 'numb_of_personal_pronouns', 'numb_of_present_tenses', 'numb_of_past_tenses', 'sent_from_web', 'numb_of_weird_chars', 'numb_of_questions', 'numb_of_emoticons', 'numb_of_swearing_words', 'numb_of_slang_words', 'numb_of_intensifiers', 'tweet_length', 'userFollowersCount', 'userFriendsCount', 'user_numb_of_tweets', 'user_list_count', 'tfidf_

In [14]:
# Load the predictions
predictions = pd.read_csv("model_results/predictions.csv")

predictions = predictions.sort_values(by=['priority'])
predictions

,priority,postCategories_x
323,0.250000,11.00
417,0.250000,11.00
418,0.250000,11.00
419,0.250000,11.00
420,0.250000,11.00
...,...,...
622,0.406667,15.90
621,0.406667,15.90
8,0.487500,18.76
7,0.487500,18.76


In [15]:
# Merge the new predictions back onto dataframe with the missing columns
df = merged_df_wo_priority.merge(predictions, left_index=True, right_index=True)
cleaned_wo = df.filter(['tweet_id','priority','postCategories_x'], axis=1)
cleaned_wo
#merged_predictions = clear_res(df)
#merged_predictions

,tweet_id,priority,postCategories_x
0,1157803414119538688,0.295000,12.44
1,1157803414119538688,0.295000,12.44
2,1157803414119538688,0.295000,12.44
3,1157806994369187840,0.262500,11.40
4,1157806994369187840,0.262500,11.40
...,...,...,...
642,1158888683552071680,0.251667,11.00
643,1158888683552071680,0.251667,11.00
644,1158890152946593792,0.250000,11.00
645,1158890152946593792,0.250000,11.00


In [16]:
# Merge the predictions back in with our labelled data
df_merged = pd.merge(cleaned_wo, merged_df, left_on = 'tweet_id', right_on = 'tweet_id', how = 'inner')

df_merged.drop_duplicates(inplace=True)

df = df_merged.groupby('tweet_id')['category'].apply(list)

df = df.to_frame()

df_merg = pd.merge(df_merged, df, left_on = 'tweet_id', right_on = 'tweet_id', how = 'inner')


df_merg

cleaned_merg = df_merg.filter(['tweet_id','eventID','priority_x','priority_y','category_x','category_y', 'postCategories_x_x','postCategories_x_y'], axis=1)
cleaned_merg


,tweet_id,eventID,priority_x,priority_y,category_x,category_y,postCategories_x_x,postCategories_x_y
0,1157803414119538688,baltimoreFlashFlood2020,0.295000,0.25,Irrelevant,[Irrelevant],12.44,11
1,1157806994369187840,baltimoreFlashFlood2020,0.262500,0.25,Irrelevant,[Irrelevant],11.40,11
2,1157807023003713536,baltimoreFlashFlood2020,0.487500,0.50,SearchAndRescue,"[SearchAndRescue, Irrelevant]",18.76,19
3,1157807023003713536,baltimoreFlashFlood2020,0.487500,0.50,Irrelevant,"[SearchAndRescue, Irrelevant]",18.76,19
4,1157813395858440192,baltimoreFlashFlood2020,0.250000,0.25,Irrelevant,[Irrelevant],11.00,11
...,...,...,...,...,...,...,...,...
214,1158881326747070464,baltimoreFlashFlood2020,0.250000,0.25,Irrelevant,[Irrelevant],11.00,11
215,1158881636840280064,baltimoreFlashFlood2020,0.250000,0.25,Irrelevant,[Irrelevant],11.00,11
216,1158883650865438720,baltimoreFlashFlood2020,0.250000,0.25,Irrelevant,[Irrelevant],11.00,11
217,1158888683552071680,baltimoreFlashFlood2020,0.251667,0.25,Irrelevant,[Irrelevant],11.00,11


In [17]:
# write to .run file
with open("marks2.run" , "w") as out_file:
    for row in df_merg.drop_duplicates(subset="tweet_id").itertuples():
        #print("row:", row)
        content = [
            eventidTopicidMap[row.eventID],
            "Q0",          
            np.int64(row.tweet_id),   
            getattr(row, 'Index'),
            str(priority_scorer[str(row.priority_y)[:3]]),
            #row.category_y,
            "['" + categoriser[round(row.postCategories_x_x,0)] + "']",
            "marksrun2"
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")